In [ ]:
%%cuda
#include <iostream>
#include <climits>

// 1 block = 8 warps = 256 threads
/*
blockIdx.x = Which Row?

blockDim.x = How many seats are in one row?

threadIdx.x = Which seat am I in right now?
*/
using namespace std;

__global__ void findMax(int *arr, int *max, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    
    if (idx < n) {
        atomicMax(max, arr[idx]);
    }
    
}

int main() {
    int n = 50;
    int a[n];
    int maxHost = INT_MIN;

    cout << "Vector elements: ";
     for (int i = 0; i < n; i++) {
        a[i] = rand() % 100;
        cout << a[i] << " ";
    }

    cout << endl;
    
    int *ad, *maxd;

    cudaMalloc(&ad, n * sizeof(int));
    cudaMalloc(&maxd, sizeof(int));

    // move data from a and maxHost to ad, maxD for GPU calculation 
    cudaMemcpy(ad, a, n * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(maxd, &maxHost, sizeof(int), cudaMemcpyHostToDevice);

    int threads = 256;
    int blocks = ( n + threads - 1) / threads;

    findMax <<< blocks, threads >>> (ad, maxd, n);

    cudaMemcpy(&maxHost, maxd, sizeof(int), cudaMemcpyDeviceToHost);
    cout << "Max element: " << maxHost << endl;

    cudaFree(ad);
    cudaFree(maxd);

    return 0;

}